In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [3]:
import pandas as pd
import numpy as np
import os

In [4]:
df = pd.read_csv('/kaggle/input/predict-student-performance-from-game-play/train.csv')
df_labels = pd.read_csv('/kaggle/input/predict-student-performance-from-game-play/train_labels.csv')

In [5]:
df= df.iloc[:5000]

In [6]:
def sep(row):
 temp = row.split('_')
 return int(temp[0])

In [7]:
df_labels['new_session_id'] = df_labels['session_id'].apply(sep)
df_labels.head(3)

,session_id,correct,new_session_id
0,20090312431273200_q1,1,20090312431273200
1,20090312433251036_q1,0,20090312433251036
2,20090314121766812_q1,1,20090314121766812


In [8]:
df_labels = df_labels.drop(columns='session_id')
df_labels.rename(columns={'new_session_id':'session_id'},inplace=True)
df_labels.head(3)

,correct,session_id
0,1,20090312431273200
1,0,20090312433251036
2,1,20090314121766812


In [9]:
df1 = pd.merge(df,df_labels,how='left', on=['session_id'])
df1.head(3)

,session_id,index,elapsed_time,event_name,name,level,page,room_coor_x,room_coor_y,screen_coor_x,...,hover_duration,text,fqid,room_fqid,text_fqid,fullscreen,hq,music,level_group,correct
0,20090312431273200,0,0,cutscene_click,basic,0,NaN,-413.991405,-159.314686,380.0,...,NaN,undefined,intro,tunic.historicalsociety.closet,tunic.historicalsociety.closet.intro,NaN,NaN,NaN,0-4,1
1,20090312431273200,0,0,cutscene_click,basic,0,NaN,-413.991405,-159.314686,380.0,...,NaN,undefined,intro,tunic.historicalsociety.closet,tunic.historicalsociety.closet.intro,NaN,NaN,NaN,0-4,1
2,20090312431273200,0,0,cutscene_click,basic,0,NaN,-413.991405,-159.314686,380.0,...,NaN,undefined,intro,tunic.historicalsociety.closet,tunic.historicalsociety.closet.intro,NaN,NaN,NaN,0-4,1


In [10]:
df1['elapsed_time_enc'] = df1['elapsed_time'].astype("category").cat.codes

In [11]:
df1['level_group_enc'] = df1['level_group'].astype("category").cat.codes

In [12]:
from sklearn.model_selection import train_test_split
x = df1[['elapsed_time_enc','level_group_enc']].to_numpy()
y = df1['correct']
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.1)
x_train[:5,:]

array([[1585,    2],
       [3632,    1],
       [4901,    1],
       [ 200,    0],
       [1224,    2]], dtype=int16)

In [14]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier()
model.fit(x_train,y_train)
y_predicted = model.predict(x_test)
from sklearn.metrics import accuracy_score, confusion_matrix
accuracy_score(y_predicted,y_test)*100

65.58888888888889